In [1]:
import pickle
import random

import scipy
import numpy as np
import pandas as pd

try:
    import SparseSC as SC
except ImportError:
    raise RuntimeError("SparseSC is not installed. Use 'pip install -e .' or 'conda develop .' from repo root to install in dev mode")

In [2]:
random.seed(12345)
np.random.seed(101101001)

In [3]:
pkl_file = "../replication/smoking_fits.pkl"

In [4]:
smoking_df = pd.read_stata("../replication/smoking.dta")
smoking_df['year'] = smoking_df['year'].astype('int')
smoking_df = smoking_df.set_index(['state', 'year'])

In [5]:
Y = smoking_df[['cigsale']].unstack('year')
T0 = 19
i_t = 2 #unit 3, but zero-index
treated_units = [i_t]
control_units = [u for u in range(Y.shape[0]) if u not in treated_units]
print(Y.shape)

(39, 31)


In [6]:
Y_names = Y.columns.get_level_values('year')
Y_pre_names = ["cigsale(" + str(i) + ")" for i in Y_names[:T0]]
print(Y.isnull().sum().sum()) #0
Y = Y.values
T = Y.shape[1]
T1 = T-T0
Y_pre,Y_post = Y[:,:T0], Y[:,T0:]

0


In [7]:
# Stata: synth cigsale beer(1984(1)1988) lnincome retprice age15to24 cigsale(1988) cigsale(1980) cigsale(1975), xperiod(1980(1)1988)  trunit(3) trperiod(1989) 

year_ind = smoking_df.index.get_level_values('year')
beer_pre = smoking_df.loc[np.logical_and(year_ind>=1984, year_ind<=1988),["beer"]]
Xother_pre = smoking_df.loc[np.logical_and(year_ind>=1980, year_ind<=1988), ['lnincome', 'retprice', 'age15to24']]
X_avgs = pd.concat((beer_pre.groupby('state').mean(), 
                    Xother_pre.groupby('state').mean())
                   , axis=1)

In [8]:
#X_spot = pd.DataFrame({'cigsale_75':smoking_df.xs(1975, level='year')["cigsale"], 
#                       'cigsale_80':smoking_df.xs(1980, level='year')["cigsale"], 
#                       'cigsale_88':smoking_df.xs(1988, level='year')["cigsale"]})
#X_orig = pd.concat((X_avgs, X_spot), axis=1)
#X_orig.isnull().sum().sum() #0

In [9]:
X_full = pd.concat((X_avgs, beer_pre.unstack('year'), Xother_pre.unstack('year')), axis=1)
X_full_names = [c[0] + "(" + str(c[1]) + ")" if len(c)==2 else c for c in X_full.columns]
X_full.isnull().sum().sum() #0
X_full = X_full.values
X_Y_pre = np.concatenate((X_full, Y_pre), axis=1)
X_Y_pre_names = X_full_names + Y_pre_names
X_Y_pre_names_arr = np.array(X_Y_pre_names)

In [10]:
def print_summary(full_fit, Y_pre, Y_post, Y_sc):
    full_Y_pre_sc,full_Y_post_sc = Y_sc[:,:T0], Y_sc[:,T0:]
    print("V: " + str(np.diag(full_fit.V)))
    print("V>0: " + str(np.diag(full_fit.V)[np.diag(full_fit.V)>0]))
    print("#V>0: " + str(sum(np.diag(full_fit.V>0))))
    full_Y_pre_effect = Y_pre - full_Y_pre_sc
    full_Y_post_effect = Y_post - full_Y_post_sc

    print("Avg bias pre: " + str(full_Y_pre_effect[control_units, :].mean()))
    print(scipy.stats.ttest_1samp(full_Y_pre_effect[control_units, :].flatten(), popmean=0)) 
    full_pre_mse = np.mean(np.power(full_Y_pre_effect[control_units, :], 2))
    print("Avg MSE pre: " + str(full_pre_mse) )

    print("Avg effect post: " + str(full_Y_post_effect[control_units, :].mean()))
    print(scipy.stats.ttest_1samp(full_Y_post_effect[control_units, :].flatten(), popmean=0)) 
    full_post_mse = np.mean(np.power(full_Y_post_effect[control_units, :], 2))
    print("Avg MSE post: " + str(full_post_mse) )

    print(X_Y_pre_names_arr[np.diag(full_fit.V)>0])

Fast

In [12]:
if False:
    fast_fit = SC.fit_fast(X_Y_pre, Y_post, treated_units=[i_t])
    #print(len(np.diag(fast_fit.V)))
    #print(np.diag(fast_fit.V))
    #Y_post_sc = fast_fit.predict(Y_post)
    #Y_pre_sc = fast_fit.predict(Y_pre)
    #post_mse = np.mean(np.power(Y_post[control_units, :] - Y_post_sc[control_units, :], 2))
    #pre_mse = np.mean(np.power(Y_pre[control_units, :] - Y_pre_sc[control_units, :], 2))
    #print(pre_mse) #192.210632448
    #print(post_mse) #129.190437803
    #print(X_Y_pre_names_arr[fast_fit.match_space_desc>0])

Full

In [12]:
use_est_fn = True
#if use_est_fn:
#    est_fit = SC.estimate_effects(
#        outcomes=Y,
#        unit_treatment_periods=X,
#        covariates=None,
#        fast = False,
#        cf_folds = len(control_units), #sync with helper
#        **kwargs
#    )
#    full_fit = 
#else: 
full_fit = SC.fit(X_Y_pre, Y_post, treated_units=[i_t], verbose=0, progress=False)



mental R^2:: 0.035163, learning rate: 0.37634,  alpha: 7.39737, zeros: 43
[STOP ITERATION: alpha is None] i: 8, grad: [ 7116440.58580422  6148010.63086454 19808390.19043673  6014178.31106925
  7209057.53591002  7160593.65541474  7120106.9391149   7068657.08419721
  7023872.99104069  6145447.33909881  6146854.74783401  6147288.1917992
  6148121.37201822  6148464.17076636  6149032.27686649  6149000.35596599
  6148981.0436949   6148918.44998959 11907623.61242215 12771335.7824121
 13854688.89545191 16957597.71550036 21625881.36699763 23631655.60836795
 24240335.24134829 27545460.09614398 33254341.26738478  6014189.71453357
  6014186.7375416   6014183.8446364   6014181.03581291  6014178.31106925
  6014175.67041432  6014173.11383363  6014170.64133792  6014168.25292671
 11572198.31839551  9284757.35135063  7993259.45809541  5812600.58213996
  4704479.54744229  3869620.39815495  2710559.15838274   817724.83691314
  2598129.26761536  5028154.74820869  4530436.91170173  7133098.93086807
  487686

In [13]:
full_Y_sc = full_fit.predict(Y)
print_summary(full_fit, Y_pre, Y_post, full_Y_sc)

V: [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 5.62581428e-06 1.67298271e-04
 1.25447705e-04 2.83218864e-05 1.20684653e-04 2.90538206e-05
 1.68471635e-04 1.91157505e-04 1.95644283e-04 2.87258604e-04
 4.25739528e-04 4.62630652e-04 6.40482407e-04]
V>0: [5.62581428e-06 1.67298271e-04 1.25447705e-04 2.83218864e-05
 1.20684653e-04 2.90538206e-05 1.68471635e-04 1.91157505e-04
 1.95644283e-04 2.87258604e-04

In [14]:
honest_predictions, cf_fits = SC.get_c_predictions_honest(X_Y_pre[control_units,:], Y_post[control_units,:], Y[control_units,:], 
                                                 w_pen=full_fit.initial_w_pen, v_pen=full_fit.initial_v_pen, cf_folds=38, verbose=1, progress=False)

 |>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

In [21]:
full_Y_sc[control_units,:] = honest_predictions
print_summary(full_fit, Y_pre, Y_post, full_Y_sc)

V: [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 5.62581428e-06 1.67298271e-04
 1.25447705e-04 2.83218864e-05 1.20684653e-04 2.90538206e-05
 1.68471635e-04 1.91157505e-04 1.95644283e-04 2.87258604e-04
 4.25739528e-04 4.62630652e-04 6.40482407e-04]
V>0: [5.62581428e-06 1.67298271e-04 1.25447705e-04 2.83218864e-05
 1.20684653e-04 2.90538206e-05 1.68471635e-04 1.91157505e-04
 1.95644283e-04 2.87258604e-04

In [20]:
full_Y_sc.shape

(39, 31)

# Full - flat
Since we don't fit v, we don't have to do out-of-sample refitting

In [15]:
full_fit_flat = SC._fit_fast_inner(X_Y_pre, X_Y_pre, Y_post, V=np.repeat(1,X_Y_pre.shape[1]), treated_units=[i_t])

In [16]:
full_flat_Y_sc = full_fit_flat.predict(Y)
print_summary(full_fit_flat, Y_pre, Y_post, full_flat_Y_sc)

V: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
V>0: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
#V>0: 55
Avg bias pre: 0.003513363039932098
Ttest_1sampResult(statistic=0.05033751319674089, pvalue=0.959867371950055)
Avg MSE pre: 3.5123625231089566
Avg effect post: -0.40206830732880827
Ttest_1sampResult(statistic=-0.7260495769207599, pvalue=0.46818168129036175)
Avg MSE post: 139.69517126549485
['beer' 'lnincome' 'retprice' 'age15to24' 'beer(1984)' 'beer(1985)'
 'beer(1986)' 'beer(1987)' 'beer(1988)' 'lnincome(1980)' 'lnincome(1981)'
 'lnincome(1982)' 'lnincome(1983)' 'lnincome(1984)' 'lnincome(1985)'
 'lnincome(1986)' 'lnincome(1987)' 'lnincome(1988)' 'retprice(1980)'
 'retprice(1981)' 'retprice(1982)' 'retprice(1983)' 'retprice(1984)'
 'retprice(1985)' 'retprice(1986)' 'retprice(1987)' 'retprice(1988)'
 'age15to24(1980)' 'age15to24(1981)' 'age15to24(1

write-out

In [22]:
#Write out
with open(pkl_file, "wb" ) as output_file:
    pickle.dump( (full_fit),  output_file) #full_fit_flat

In [11]:
#Read back
with open(pkl_file, "rb" ) as input_file:
    (full_fit) = pickle.load(input_file)